In [ ]:
import evaluate
import datasets
import tqdm
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from utils import eval_score
import pandas as pd

rouge = evaluate.load('rouge')

base_dir = '/var/scratch/mca305'
model_name = 'ROBERTA_BASE'
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-base", output_hidden_states=True)

embedding_dir = f'embeddings_{model_name}'
data_dir = f'data_{model_name}'

eval_methods = {
    'Method': [],
    'Summary': [],
    'Label': [],
    'rouge1': [],
    'rouge2': [],
    'rougeL': [],
    'rougeLsum': []
}
cnt = 0

small_DS = datasets.load_from_disk(f"{base_dir}/{data_dir}")
#small_DS = cnn_data_validation.filter(lambda x: x['id'] in small_DS['article_id'])
# TODO remove ?
#small_DS = small_DS.map(split_in_sents, remove_columns=small_DS.column_names, batched=True)
small_DS = datasets.load_from_disk(f"{base_dir}/{embedding_dir}")

print('Embeddings extracted... now calculating STs', flush=True)
unique_ids = list(set(small_DS['article_id']))
for u_id in tqdm(unique_ids):
    current_ds = small_DS.filter(lambda x: x['article_id'] == u_id)

    sents = current_ds['sentence']
    summary = current_ds['highlight'][0]

    # Eval scores BERT
    k_center, stl_k_center, random_e, k_center_trim, summaries = eval_score(sents, current_ds['embeddings'], summary, rouge)

    # k_center
    eval_methods['Method'].append('K Center')
    eval_methods['Summary'].append(summaries[0])
    eval_methods['Label'].append(summary)
    for k in list(k_center.keys()):
        eval_methods[k].append(k_center[k])

    # st_k_center
    eval_methods['Method'].append('Salient STL')
    eval_methods['Summary'].append(summaries[1])
    eval_methods['Label'].append(summary)
    for k in list(stl_k_center.keys()):
        eval_methods[k].append(stl_k_center[k])

    # random
    eval_methods['Method'].append('Random')
    eval_methods['Summary'].append(summaries[2])
    eval_methods['Label'].append(summary)
    for k in list(random_e.keys()):
        eval_methods[k].append(random_e[k])

    # K Center Trimmed
    eval_methods['Method'].append('K Center STL')
    eval_methods['Summary'].append(summaries[3])
    eval_methods['Label'].append(summary)
    for k in list(k_center_trim.keys()):
        eval_methods[k].append(k_center_trim[k])
    cnt += 1

    ## Eval scores ROBERTA
    print(f'{cnt} out of {len(unique_ids)}', flush=True)

df = pd.DataFrame.from_dict(eval_methods)
df.to_csv(f'scoring_{model_name}.csv')